In [ ]:
from __future__ import absolute_import, division, print_function
%matplotlib inline

import glob 
import os
import numpy as np
import PIL.Image as pil
from PIL import ImageDraw, ImageFont
import matplotlib.pyplot as plt

import matplotlib as mpl
import matplotlib.cm as cm

import torch
from torchvision import transforms
import networks
from utils import readlines
from collections import OrderedDict

from thop import profile, clever_format
import copy
import cv2
from random import shuffle
import time

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
no_cuda = False
data_path = "/work/garin0115/datasets/kitti_data/"
split = "eigen"
image_path = ""
ext = "jpg"
split_folder = os.path.join(os.path.expanduser("~"), "depth", "monodepth2", "splits", split)
CMAP = 'plasma'

In [ ]:
if torch.cuda.is_available() and not no_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
# encoder_dict = {"shufflenetv2":networks.ShuffleNetV2(), 
# #                 "shufflenetv2_more":networks.ShuffleNetV2(),
#                 "mobilenetv2":networks.MobileNetV2(), 
# #                 "mobilenetv3_more":networks.MobileNetV3(),
#                 "mobilenetv3":networks.MobileNetV3(),
#                 "mnasnet":networks.MnasNet(),
#                 "peleenet":networks.PeleeNet(),
#                 "resnet18":networks.ResnetEncoder(18, False),
#                 "resnet50":networks.ResnetEncoder(50, False),
# #                 "peleenet_share":networks.PeleeNet()
#                 }
# encoder_dict = {
#                 "resnet50":networks.ResnetEncoder(50, False),
#                 "mnasnet":networks.MnasNet(),
#                 "mnasnet_share":networks.MnasNet(),
#                 "shufflenetv2":networks.ShuffleNetV2(), 
#                 "shufflenetv2_share":networks.ShuffleNetV2(),
#                 "mobilenetv3":networks.MobileNetV3(), 
#                 "mobilenetv3_share":networks.MobileNetV3(),
#                 "mobilenetv2":networks.MobileNetV2(),
#                 "mobilenetv2_share":networks.MobileNetV2(),
#                 "peleenet":networks.PeleeNet(),
#                 "peleenet_MJ":networks.PeleeNet(),
#                 "peleenet_share":networks.PeleeNet(),
#                 "peleenet_1x1_MJ":networks.PeleeNet(),
#                 "peleenet_posecnn":networks.PeleeNet(),
#                 "peleenet_share_1x1_MJ":networks.PeleeNet(),
#                 }
    
encoder_dict = {"resnet18":networks.ResnetEncoder(18, False)}
# decoder_dict = {"resnet18":networks.DepthDecoder(encoder_dict["resnet18"].num_ch_enc)}
# decoder_dict = {"resnet18":networks.DepthDecoder(encoder_dict["resnet18"].num_ch_enc),
#                 "resnet18_nn3":networks.NNDecoder(
#                     encoder_dict["resnet18"].num_ch_enc, kernel_size=3, dw=False),
#                 "resnet18_nn3_dw":networks.NNDecoder(
#                     encoder_dict["resnet18"].num_ch_enc, kernel_size=3, dw=True),
#                 "resnet18_nn5":networks.NNDecoder(
#                     encoder_dict["resnet18"].num_ch_enc, kernel_size=5, dw=False),
#                 "resnet18_nn5_dw":networks.NNDecoder(
#                     encoder_dict["resnet18"].num_ch_enc, kernel_size=5, dw=True),
#                 "resnet18_nn35":networks.NNDecoder(
#                     encoder_dict["resnet18"].num_ch_enc, kernel_size=35, dw=False),
#                 "resnet18_nn53":networks.NNDecoder(
#                     encoder_dict["resnet18"].num_ch_enc, kernel_size=53, dw=False),
#                 "resnet18_nn3_3x3":networks.NNDecoder(
#                     encoder_dict["resnet18"].num_ch_enc, kernel_size=3, dw=False, pw=3),
#                 "resnet18_nn3_upconv":networks.NNDecoder(
#                     encoder_dict["resnet18"].num_ch_enc, kernel_size=3, dw=False, upconv=True)}

# decoder_dict = {"resnet18":networks.DepthDecoder(encoder_dict["resnet18"].num_ch_enc),
# #                 "resnet18_nn3":networks.NNDecoder(
# #                     encoder_dict["resnet18"].num_ch_enc, kernel_size=3, dw=False),
#                 "resnet18_my3_dw":networks.MYDecoder(
#                     encoder_dict["resnet18"].num_ch_enc, kernel_size=3, dw=True),
# #                 "resnet18_nn5":networks.NNDecoder(
# #                     encoder_dict["resnet18"].num_ch_enc, kernel_size=5, dw=False),
# #                 "resnet18_nn5_dw":networks.NNDecoder(
# #                     encoder_dict["resnet18"].num_ch_enc, kernel_size=5, dw=True),
#                 "resnet18_my35":networks.MYDecoder(
#                     encoder_dict["resnet18"].num_ch_enc, kernel_size=35, dw=True),
# #                 "resnet18_nn53":networks.NNDecoder(
# #                     encoder_dict["resnet18"].num_ch_enc, kernel_size=53, dw=False),
#                 "resnet18_my3_3x3":networks.MYDecoder(
#                     encoder_dict["resnet18"].num_ch_enc, kernel_size=3, dw=False, pw=3),
#                 "resnet18_my3_3x3_dw":networks.MYDecoder(
#                     encoder_dict["resnet18"].num_ch_enc, kernel_size=3, dw=True, pw=3)}

# decoder_dict = {"resnet18":networks.DepthDecoder(encoder_dict["resnet18"].num_ch_enc),
#             "resnet18_my3":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc),
                
#             "resnet18_my35_moreFeature":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc, moreFeature=True, kernel_size=35),
#             "resnet18_my3_more1":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc, moreConv=1),
#             "resnet18_my3_more2":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc, moreConv=2),
                
#             "resnet18_my3_doubleConv1":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc, doubleConv=1),
#             "resnet18_my3_doubleConv3":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc, doubleConv=3),
#             "resnet18_my3_doubleConv3_3x3":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc, doubleConv=3, pw=False),
                
#             "resnet18_my3_firstConv":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc, firstConv=True),
#             "resnet18_my3_skipSky":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc),
#             "resnet18_my3_firstConv_skipSky":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc, firstConv=True),
                
#             "resnet18_my3_firstConv_skipSky_conv11":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc, firstConv=True, conv11=True),
#             "resnet18_my3_smooth":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc),
#             "resnet18_my3_concatDepth":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc, concatDepth=True),
#             }

decoder_dict = {"resnet18":networks.DepthDecoder(encoder_dict["resnet18"].num_ch_enc),
            "resnet18_my3":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc),
                
            "resnet18_my3_firstConv":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc, firstConv=True),
            "resnet18_my3_skipSky":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc),
            "resnet18_my3_firstConv_skipSky":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc, firstConv=True),
                
            "resnet18_my3_firstConv_skipSky_conv11":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc, firstConv=True, conv11=True),
            "resnet18_my3_smooth":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc),
            "resnet18_my3_concatDepth":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc, concatDepth=True),
            
            "resnet18_skipFirstConv":networks.DepthDecoder(encoder_dict["resnet18"].num_ch_enc, skipFirstConv=True),
            "resnet18_skipFirstConv_skipSky":networks.DepthDecoder(encoder_dict["resnet18"].num_ch_enc, skipFirstConv=True),
            "resnet18_my3_firstConv_skipSky":networks.MYDecoder(encoder_dict["resnet18"].num_ch_enc, firstConv=True),
            }

In [ ]:
models = {}
lines = readlines(os.path.join(split_folder, "test_files.txt"))
side_map = {"2": 2, "3": 3, "l": 2, "r": 3}

inWork = ["resnet18", "resnet18_my3", "resnet18_my35_moreFeature", "resnet18_my3_more1", "resnet18_my3_more2", "resnet18_my3_concatDepth", "resnet18_my3_doubleConv1", "resnet18_my3_doubleConv3"]

with torch.no_grad():
    for name in decoder_dict:
        print("Deal with ", name)
        if name in inWork:
            weight_path = os.path.join("/work",
                                      "garin0115",
                                      "models",
                                      name+"_256x832",
                                      "models",
                                      "weights_19")
        else:
            weight_path = os.path.join(os.path.expanduser("~"), 
                                       "depth",
                                       "monodepth2",
                                       "models", 
                                       name+"_256x832", 
                                       "models", 
                                       "weights_19")
            
        encoder_path = os.path.join(weight_path , "encoder.pth")
        depth_decoder_path = os.path.join(weight_path, "depth.pth")


        encoder = encoder_dict["resnet18"]
        depth_decoder = decoder_dict[name]

        loaded_dict_enc = torch.load(encoder_path, map_location='cpu')
        filtered_dict_enc = {k: v for k, v in loaded_dict_enc.items() if k in encoder.state_dict()}
        loaded_dict = torch.load(depth_decoder_path, map_location='cpu')

        models[name+"_encoder"] = copy.deepcopy(encoder)
        models[name+"_decoder"] = copy.deepcopy(depth_decoder)
        models[name+"_height"] = loaded_dict_enc["height"]
        models[name+"_width"] = loaded_dict_enc['width']

        models[name+"_encoder"].load_state_dict(filtered_dict_enc)
        models[name+"_decoder"].load_state_dict(loaded_dict)

        models[name+"_encoder"].to(device)
        models[name+"_decoder"].to(device)
        models[name+"_encoder"].eval()
        models[name+"_decoder"].eval()

#             input_image_resized = input_image.resize((models[name+"_width"], models[name+"_height"]), pil.LANCZOS)
#             input_image_torch = transforms.ToTensor()(input_image_resized).unsqueeze(0)
#             input_image_torch = input_image_torch.to(device)

#             features = models[name+"_encoder"](input_image_torch)
#             outputs = models[name+"_decoder"](features)

#             disp = outputs[("disp", 0)]
#             disp_resized = torch.nn.functional.interpolate(
#                     disp, (original_height, original_width), mode="bilinear", align_corners=False)
#             disp_resized_np = disp_resized.squeeze().cpu().detach().numpy()

#             show["Input"] = input_image
#             show["{}_disp".format(name)] = disp_resized_np


#         plt.figure(figsize=(20, 18)) 

#         for idx, key in enumerate(show):
#             if idx == 0:
#                 plt.subplot(621)
#                 plt.imshow(show[key])
#                 plt.title("Input", fontsize=22)
#                 plt.axis('off')
#             else:
#                 plt.subplot(6, 2, idx+1)
#                 vmax = np.percentile(show[key], 95)
#                 plt.imshow(show[key], cmap=CMAP, vmax=vmax)
#                 plt.title(key, fontsize=22)
#                 plt.axis('off')
#         plt.tight_layout(pad=0.5, w_pad=0.1, h_pad=0.1)

# For test_files.txt 

In [ ]:
lines = readlines(os.path.join(split_folder, "test_files.txt"))
side_map = {"2": 2, "3": 3, "l": 2, "r": 3}
with torch.no_grad():
    for i in np.random.choice(len(lines), 10, replace=False):
        folder, frame_id, side = lines[i].split()
        frame_id = int(frame_id)        
        

        image_path = os.path.join(data_path, folder, "image_0{}".format(side_map[side]), "data", "{:010d}.jpg".format(frame_id))
#         image_path = "assets/test_image.jpg"
        input_image = pil.open(image_path).convert('RGB')
        original_width, original_height = input_image.size
        
        show = OrderedDict()
        show["Input"] = input_image

        
        for name in decoder_dict:
            input_image_resized = input_image.resize((models[name+"_width"], models[name+"_height"]), pil.LANCZOS)
            input_image_torch = transforms.ToTensor()(input_image_resized).unsqueeze(0)
            input_image_torch = input_image_torch.to(device)
            
            features = models[name+"_encoder"](input_image_torch)
            outputs = models[name+"_decoder"](features)

            disp = outputs[("disp", 0)]
            disp_resized = torch.nn.functional.interpolate(
                    disp, (original_height, original_width), mode="bilinear", align_corners=False)
            disp_resized_np = disp_resized.squeeze().cpu().detach().numpy()
            show["{}_disp".format(name)] = disp_resized_np
            
            
        plt.figure(figsize=(25, 15)) 
        for idx, key in enumerate(show):
            if idx == 0:
                plt.subplot(431)
                plt.imshow(show[key])
                plt.title("Input", fontsize=22)
                plt.axis('off')
            else:
                plt.subplot(4, 3, idx+1)
                vmax = np.percentile(show[key], 95)
                plt.imshow(show[key], cmap=CMAP, vmax=vmax)
                plt.title(key, fontsize=22)
                plt.axis('off')
        plt.tight_layout(pad=0.5, w_pad=0.1, h_pad=0.1)

# For video

In [ ]:
# 選擇要建立 video 的 data [TODO]
# file_name = '2011_10_03/2011_10_03_drive_0047_sync' #837
# file_name = '2011_09_30/2011_09_30_drive_0016_sync' #279
# file_name = '2011_09_29/2011_09_29_drive_0026_sync' #158
# file_name = '2011_09_28/2011_09_28_drive_0037_sync' #89
file_name = '2011_09_26/2011_09_26_drive_0036_sync' #803
# file_name = '2011_09_26/2011_09_26_drive_0023_sync' #474
# file_name = '2011_09_26/2011_09_26_drive_0020_sync' #86
# file_name = '2011_09_26/2011_09_26_drive_0013_sync' #144
# file_name = '2011_09_26/2011_09_26_drive_0002_sync' #77

# 選擇影片輸出資料夾 [TODO]
video_output_folder = os.path.join(os.path.expanduser("~"), 
                                                      "depth",
                                                      "monodepth2",
                                                      "video_result")

# 取得資料夾中所有影像檔案路徑
kitti_depth_folder = '/work/garin0115/datasets/kitti_data/'+file_name+'/image_02'
filenames = glob.glob(kitti_depth_folder+'/*/*.jpg')

# 將檔案路徑排序
filenames.sort()
num_images = len(filenames)
print("Total images: {}".format(num_images))

In [ ]:
show = OrderedDict()
show["Input"] = []
for img in filenames:
    input_image = pil.open(img).convert('RGB')
    show["Input"].append(input_image)

with torch.no_grad():
    for name in decoder_dict:
        print("Deal with {}".format(name))
        show["{}_disp".format(name)] = []
        show["{}_FPS".format(name)] = []
        for input_image in show["Input"]:
            
            original_width, original_height = input_image.size

            input_image_resized = input_image.resize((models[name+"_width"], models[name+"_height"]), pil.LANCZOS)
            input_image_torch = transforms.ToTensor()(input_image_resized).unsqueeze(0)
            input_image_torch = input_image_torch.to(device)
            
            
            start_time = time.time()
            features = models[name+"_encoder"](input_image_torch)
            outputs = models[name+"_decoder"](features)
            end_time = time.time()

            disp = outputs[("disp", 0)]
            disp_resized = torch.nn.functional.interpolate(
                    disp, (original_height, original_width), mode="bilinear", align_corners=False)
            disp_resized_np = disp_resized.squeeze().cpu().numpy()
            
            show["{}_disp".format(name)].append(disp_resized_np)
            show["{}_FPS".format(name)].append("{:.2f}".format(1/(end_time-start_time)))


In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(video_output_folder+'/disp_{}.avi'.format(file_name.split('/')[-1]), fourcc, 15.0, (original_width*3, original_height*5))
num_model = len(decoder_dict)

for idx in range(num_images):
    res = []
    input = np.array(show["Input"][idx])
    cv2.putText(input, "Input", (10, 40), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (255, 255, 255), 2, cv2.LINE_AA)
    res.append(input[:, :, ::-1])
    
    for name in decoder_dict:
        disp = show["{}_disp".format(name)][idx]
        vmax = np.percentile(disp, 95)
        normalizer = mpl.colors.Normalize(vmin=disp.min(), vmax=vmax)
        mapper = cm.ScalarMappable(norm=normalizer, cmap=CMAP)
        colormapped_im = (mapper.to_rgba(disp)[:, :, :3] * 255).astype(np.uint8)
        cv2.putText(colormapped_im, name, (10, 40), cv2.FONT_HERSHEY_TRIPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(colormapped_im, "FPS "+show["{}_FPS".format(name)][idx], (10, 100), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0, 255, 0), 2, cv2.LINE_AA)
        im = pil.fromarray(colormapped_im[:, :, ::-1])
        res.append(im)

    result = np.vstack([np.hstack(res[:3]), np.hstack(res[3:6]), np.hstack(res[6:9]), 
                        np.hstack(res[9:12]), np.hstack(res[12:])])

        
    out.write(result)
out.release()
    
            

# Test Pose 

In [ ]:
prev_img = None
orig = np.array([0, 0, 0, 1])
pos_gt = [orig.copy()]
pos_pred = [orig.copy()]

# seg = 'road04_seg'
# record = 'Record021'
# record_dir = '/home/ubuntu/hdd/data/Apolloscape/Segmentation/%s/ColorImage/%s/Camera 5'%(seg, record)
# vid_name = '%s_%s.avi'%(seg, record)

# record_dir = '/home/ubuntu/hdd/data/kitti/RawData/2011_09_30/2011_09_30_drive_0033_sync/image_02/data/'
# vid_name = '2011_09_30_drive_0033_2.avi'

record_dir = '/home/ubuntu/hdd/data/Cityscapes/leftImg8bit_demoVideo/leftImg8bit/demoVideo/stuttgart_02/'
vid_name = 'stuttgart_02.avi'

# apollo pose
# pose_gt = pd.read_csv('/home/ubuntu/hdd/data/Apolloscape/Segmentation/%s/Pose/%s/Camera 5/pose.txt'%(seg, record), sep=' ', header=None)
# pose_gt.sort_values(16, inplace=True)
# pose_gt.drop(pose_gt.columns[16], axis=1, inplace=True)
# pose_gt = np.array(pose_gt, dtype=np.float32).reshape(-1, 4, 4)

# kitti pose
# pose_gt = pd.read_csv('/home/ubuntu/hdd/data/kitti/odometry/poses/09.txt', sep=' ', header=None)
# pose_gt[12] = 0
# pose_gt[13] = 0
# pose_gt[14] = 0
# pose_gt[15] = 1
# pose_gt = np.array(pose_gt, dtype=np.float32).reshape(-1, 4, 4)

out = cv2.VideoWriter(vid_name, cv2.VideoWriter_fourcc(*'XVID'), 10.0, (feed_width*2+320,feed_height))
T_pred = []
T_gt = np.eye(4)
T_pr = np.eye(4)
for i, filename in enumerate(sorted(os.listdir(record_dir))):
    image_path = os.path.join(record_dir, filename)
    input_image = pil.open(image_path).convert('RGB')
    input_image_resized = input_image.resize((feed_width, feed_height), pil.LANCZOS)
    input_image_pytorch = transforms.ToTensor()(input_image_resized).unsqueeze(0).cuda()
    
    with torch.no_grad():
        features = encoder(input_image_pytorch)
        outputs = depth_decoder(features)
        disp = outputs[("disp", 0)][0]
        if prev_img is not None:
            features_pose = [pose_encoder(torch.cat([prev_img, input_image_pytorch], 1))]
            axisangle, translation = pose_decoder(features_pose)
            T_ = transformation_from_parameters(axisangle[:, 0], translation[:, 0], True).cpu().numpy()[0]
            T_pr = T_pr.dot(T_)
            T_pred += [T_]
#             T_gt = T_gt.dot(np.linalg.inv(pose_gt[i-1]).dot(pose_gt[i]))
#             pos_gt += [T_gt[:,-1]]
            pos_pred += [T_pr[:,-1]]
    
    disp_np = disp.squeeze().cpu().numpy()
    vmax = np.percentile(disp_np, 95)
    output = np.zeros((feed_height, feed_width*2+320, 3), dtype=np.uint8)
    buf = io.BytesIO()
    plt.imsave(buf, disp_np, cmap='plasma', vmax=vmax)
    buf.seek(0)
    disp_ = np.array(pil.open(buf))[:,:,:3]
    buf.close()
    
    pos_gt_ = np.array(pos_gt)
    plt.figure(figsize=(5, 4))
    plt.title("Visual odometry", fontsize=15)
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='datalim')
    ax.spines['left'].set_position('zero')
    ax.spines['bottom'].set_position('zero')
#     plt.plot(pos_gt_[:, 0], pos_gt_[:, 2], 'o-', label='gt')
    scale = 29.5
    pos_pred_ = np.array(pos_pred)*scale
    plt.plot(pos_pred_[:, 0], pos_pred_[:, 2], 'o-', label='pred')
    plt.scatter(pos_pred_[-1, 0], pos_pred_[-1, 2], color='r', s=100, zorder=10)
    plt.text(pos_pred_[-1, 0], pos_pred_[-1, 2], 'NOW', fontsize=15, zorder=20)
    plt.legend(loc=4, fontsize=15)
    buf = io.BytesIO()
    plt.savefig(buf, dpi=64)
    plt.close()
    buf.seek(0)
    vo = np.array(pil.open(buf))[:,:,:3]
    buf.close()
    
    output[:,:feed_width] = np.array(input_image_resized)
    output[:,feed_width:-320] = disp_
    output[:,-320:] = vo
    out.write(output[:,:,::-1])
    
    prev_img = input_image_pytorch
    print('\r', i+1, len(os.listdir(record_dir)), end=' '*10)
    
out.release()

In [ ]:
pos_gt = [orig.copy()]
pos_pred = [orig.copy()]

T_gt = np.eye(4)
T_pr = np.eye(4)
for i in range(1, len(pose_gt)):
    T_pr = T_pr.dot(T_pred[i-1])
    T_gt = T_gt.dot(np.linalg.inv(pose_gt[i-1]).dot(pose_gt[i]))
    pos_gt += [T_gt[:,-1]]
    pos_pred += [T_pr[:,-1]]
    
pos_gt = np.array(pos_gt)
plt.figure(figsize=(5, 8))
plt.title("Visual odometry", fontsize=15)
ax = plt.gca()
ax.set_aspect('equal', adjustable='datalim')
plt.plot(pos_gt[:, 0], pos_gt[:, 2], '-o', label='gt')
scale = 34.169
pos_pred = np.array(pos_pred)*scale
plt.plot(pos_pred[:, 0], pos_pred[:, 2], '-o', label='pred')
plt.scatter(pos_pred[-1, 0], pos_pred[-1, 2], color='r', s=100, zorder=10)
plt.text(pos_pred[-1, 0], pos_pred[-1, 2], 'NOW', fontsize=15, zorder=20)
plt.legend(loc=4, fontsize=15)

buf = io.BytesIO()
plt.savefig(buf, dpi=64)
# plt.close()
buf.seek(0)
x = np.array(pil.open(buf))[:,:,:3]
buf.close()